In [22]:
import sys
sys.path.append('..')

In [23]:
from pathlib import Path

In [24]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [25]:
from models import Model
from utils.config import Config

In [53]:
MODEL_FILE = '/home/jiun/datasets/weights/nvr-warp/ssd-vgg16/SSD-300000.pth'

In [54]:
num_classes = 2
batch = 1

In [55]:
network = 'SSD'
backbone = 'VGG16'

In [56]:
MODEL = Model.get(network).get(backbone)

In [57]:
config = Config('', network, MODEL)
config.nms = False

Configfile  is not exists or can not open


In [46]:
config = Config('../configs/aspect-ratio.json', network, MODEL)
config.nms = False

In [58]:
device = torch.device('cuda:0')

In [59]:
model = MODEL.new(num_classes, batch, config)
model.to(device)

VGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [60]:
model.load(torch.load(MODEL_FILE, map_location=lambda s, l: s))
model.eval()

VGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [61]:
inputs = Variable(torch.ones(1, 3, *config.size), requires_grad=False).to(device)
outputs = model(inputs)

print(outputs.min(), outputs.max())
print(outputs.mean(), outputs.std())

module = torch.jit.trace(model, inputs)
torch.jit.save(module, 'ssd-vgg.pt')

tensor(-0.4252, device='cuda:0') tensor(1.6559, device='cuda:0')
tensor(0.5067, device='cuda:0') tensor(0.3394, device='cuda:0')


In [19]:
module = torch.jit.load('ssd-vgg-aspect.pt')

In [18]:
output = module(inputs)

In [19]:
print(output.min(), output.max())
print(output.mean(), output.std())

tensor(-0.5691, grad_fn=<MinBackward1>) tensor(2.1194, grad_fn=<MaxBackward1>)
tensor(0.5045, grad_fn=<MeanBackward0>) tensor(0.3507, grad_fn=<StdBackward0>)


In [ ]:
show(img.astype(np.uint8))

In [ ]:
from PIL import Image
from IPython.display import display

def show(ary):
    display(Image.fromarray(ary))